In [ ]:
from bs4 import BeautifulSoup
import requests 
import datetime
import re
import csv
from csv import writer
import pandas as pd
import time

In [ ]:
# initialise empty lists 
comments_csv = []
comment_date_csv = []
userid_csv = []

#find last page number
page_url = '<YOUR URL LINK>'
page = requests.get(page_url)

soup = BeautifulSoup(page.text, 'html.parser')

results = soup.find_all("input", {"class": "input input--number js-numberBoxTextInput input input--numberNarrow js-pageJumpPage"})
lastPage = (results[0]['max'])

# Extracting review texts, datetime and userid
for x in range(1,int(lastPage)+1):
    hardware_zone_url = '<YOUR URL LINK>' #url link with removed page number 
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    response = requests.get(hardware_zone_url + str(x))
    soup1 = BeautifulSoup(response.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    all_comments = soup2.find_all('div',class_='bbWrapper')
    all_dates = soup2.find_all('li', class_='u-concealed')
    userid = soup2.find_all('h4', class_='message-name')
    all_links = soup2.find_all('a', class_="link link--external fauxBlockLink-blockLink")
    all_links_content = soup2.find_all('div', class_='contentRow-snippet js-unfurl-desc')
    all_links_content2 = soup2.find_all('div', class_='contentRow-minor contentRow-minor--hideLinks')
    all_links_reviewer_name = soup2.find_all('a', class_='bbCodeBlock-sourceJump')
    all_links_reviewer_comment = soup2.find_all('div', class_='bbCodeBlock-expandContent js-expandContent')
    all_links_expand = soup2.find_all('div', class_='bbCodeBlock-expandLink js-expandLink')
    all_links_external = soup2.find_all('a', class_='link link--external')

    #remove unwanted information (e.g. links and other texts not commented by reviewer)
    for i in all_links:
        i.decompose()

    for i in all_links_content:
        i.decompose()

    for i in all_links_content2:
        i.decompose()

    for i in all_links_reviewer_name:
        i.decompose()

    for i in all_links_reviewer_comment:
        i.decompose()

    for i in all_links_expand:
        i.decompose()

    for i in all_links_external:
        i.decompose()

    for i in all_comments:
            comments = i.get_text()
            comments2 = comments.strip()
            comments3 = ' '.join(re.split(r'[ \n\t]+',comments2))
            print(comments3)
            comments_csv.append(comments3)

    for i in all_dates:
        all_dates1 = i.find('time')
        if all_dates1.has_attr('datetime'):
          print(all_dates1['datetime'])
          comment_date_csv.append(all_dates1['datetime'])

    for i in userid:
        userid1 = i.find('a')
        if userid1.has_attr('data-user-id'):
            print(userid1['data-user-id'])
            userid_csv.append(userid1['data-user-id'])

    time.sleep(15)


In [ ]:
#remove first comment of review and post 
def delete_first_comment():
    del comments_csv[0]
    del comment_date_csv[0]
    del userid_csv[0]

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re  
import spacy
nlp = spacy.load('en_core_web_lg')
#python -m spacy download en
#python -m spacy download en_core_web_lg

In [ ]:
list = []
list_nopunct = []

#split sentence into individual words
for line in comments_csv:
    words = line.split()
    list.append(words)

#remove punctuations 
for i in list:
    no_punct = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in i]
    list_nopunct.append(no_punct)

#remove blanks ('')
list_nopunct2 = [[i for i in nested if i != ''] for nested in list_nopunct]

#stem words
s_stemmer = SnowballStemmer(language='english')
stem_comments = []
for i in list_nopunct2:
    stem = [(s_stemmer.stem(x)) for x in i]
    stem_comments.append(stem)
    print(stem_comments)
    
#remove all stop words 
stop_comments = []
stop_comments = [[i for i in nested if i not in nlp.Defaults.stop_words] for nested in stem_comments]

In [ ]:
#write every row into csv file, for export

header = ['comment_word', 'userid', 'date_of_comment']

with open('plh_tenyear_mop.csv', 'a', newline='') as f_object:
    writer_object = writer(f_object)
    writer_object.writerow(header)
    
    indexCount = len(userid_csv)
    initValue = 0 
    for user in userid_csv:
        for comment in stop_comments[initValue]:
            #print(comment, user,comment_date_csv[initValue])
            writer_object.writerow([comment, user,comment_date_csv[initValue]])
        initValue+=1

    f_object.close()

In [ ]:
#count total number of items in nested list 
#to double confirm that number of rows in excel is same
count = 0
for listElem in list_nopunct2:
    count += len(listElem)   

print(count)